## YOLO V1 Bounding Box
- (1) 예측하고자 하는 이미지를 SxS Grid Cell로 나눔
- (2) 각 cell 마다 하나의 객체를 예측
    - 객체가 겹쳐 있으면 몇몇의 객체는 탐지를 못한다. (하나만 탐지하니까)
- (3) B개의 Boundary Boxes를 예측 및 각 box는 하나의 box confidence score를 가짐.
    - Confidence Score는 box가 **객체를 포함하고 있을 가능성(objectness)**와 **boundary box의 정확성**을 반영 
    - 하나의 Boundary box가 그려지면 그 안에 0,1,2등의 다양한 객체 있을텐데, 각자 종합.
    - **객체 존재시 Objectness 올라가고, boundary box에 대한 정확성?(아마 IOU 인듯) 구함**
- (4) 예측된 box 수에 관계없이 단 하나의 객체만 탐지
- (5) C개의 conditional class probabilities(탐지된 객체가 어느 특정 클래스에 속할까)를 예측
    - truck:0.3, person:0.10, car:0.5 ...
- Result
    - 각 boundary box는 객체의 위치(x,y), 객체의 크기(w,h), box confidences score로 총 5개의 인자를 가짐
    - output
        - 7x7 grids, 2 boundary boxes, 20 clasees 
        - 49개의 각 cell에 대해 2 x (x, y, w, h, conf score) + 20 = 30개씩의 결과값이 필요
        - 즉 최종 output 텐서는 (7, 7, 30)

<hr>

## YOLO V2,9000 Bounding Box
- YOLO 9000은 Anchor Box와 Reference Center Point의 Shift값인 tx, ty, tw, th를 예측하고 bx, by, bw, bh값을 구해서 L2 LOSS를 통해서 학습을 진행

- 성능향상의 주요 요소들 
- (1) BatchNormalization 
    - BN을 통한 학습속도 및 정확도 향상
- (2) Convolutional with Anchor
    - 사전에 정의된 크기의 Boxes를 설정하여 초기 학습을 안정화 (anchor의 도입)
- (3) Remove fully-connected layers
    - SSD는 FC를 없애고 Conv를 사용하였는데 YOLO V2도 해당 구조로 변경
    - 즉 Class확률을 CNN에서 뽑아낸다는 점.
    - ex) class수가 20개일경우
        - 각 cell의 출력 parameter는 25개임 (x,y,w,h,conf score) + 20class
        - anchor boxes가 5개일경우 => 1cell기준 5*25 = 125개의 parameter가 출력
- (4) 네트워크 구조 변경
    - 7x7이 아닌 13x13 grid로 변경
    - 작은 Object를 탐지하기 위한 passthrough 방식 도입
    
<hr>


## YOLO V3 Bounding Box

![yolo_v3_boundingbox](img/yolo_v3_boundingbox.jpg)

- 쉽게 말해서 c_x, c_y가 존재(현재 위치느낌), yolo가 클러스터링을 통해서 anchor box를 생성함.
- 1) c_x, c_y : grid 셀의 좌상단의로부터의 좌표(offset)
    - YOLO는 이미지를 S*S grid 셀로 나눔
    - 예를 들어 7*7 grid 셀로 설정하면, 한 이미지에 7*7 = 49개의 셀이 생성

- 2) p_w, p_h
    - 앵커박스의 너비 및 높이
- 3) t_x, t_y, t_w, t_h
    - YOLO가 예측해야하는 물체의 좌표 값 (bounding box)
    - 시그모이드 함수를 통한 t_x, t_y를 0~1 사이의 값으로 초기화
- 4) b_x, b_y, b_w, b_h
    - 좌상단 offset으로 부터 얼마나 이동할것이고 (b_x, b_y)
    - 앵커박스의 너비와 높이를 얼만큼 비율로 조절할지 (b_w, b_h) 지수승을 통해 예측
    - 위 파라미터값을 조정한 최종 bounding box의 offsets 값들
    - 이를 토대로 GT(Ground Truth, 라벨 값)와 IOU를 계산

- 각각의 예측된 box마다 Objectness에 대한 점수를 logistic regression을 통해 구함
    - 즉 만약 예측된 어떤 box가 다른 box들보다 IOU가 높다면 해당값은 1
    - 만약 IOU가 제일 좋지는 않지만 특정한 Threshold보다 높다면 Faster R-CNN 방법을 사용해 무시
- YOLO v3는 IOU Threshold값을 0.5를 사용
- 하나의 box에 하나의 GT Box를 할당, box가 GT에 할당되지 않으면 좌표 및 클래스 예측에 손실을 주지 않고, Objectness에 대한 손실값만 적용

### Faster R-CNN Anchor Box vs Yolo v3 Anchor Box
- Faster R-CNN
    - 특정 Anchor가 Ground Truth와의 IOU가 0.7보다 크면 -> 해당 anchor box를 positive
    - 특정 Anchor가 Ground Truth와의 IOU가 0.3보다 작으면 -> 해당 anchor box는 non-positive
    - 나머지 구간 -> IOU가 가장 큰 anchor box를 positive로
    
- Yolo V3 
    - 특정 Anchor가 Ground Truth와의 IOU가 0.5보다 크면 -> 해당 Anchor Box를 Positive
    - 특정 Anchor가 Ground Truth와의 IOU가 0.5보다 작으면 -> 해당 경우 무시 및 Objectness에 대한 손실값 적용

  - b_x, b_y : 시그모이드 함수(sigmoid function)를 통해 t_x, t_y를 0~1 사이의 값으로 초기화

## 참고문헌
- https://ropiens.tistory.com/43
- https://wdprogrammer.tistory.com/50
- https://dhhwang89.tistory.com/138